### <center> Testes de Implementação: **Ollama e Langchain** em Agentes Inteligentes <center>


Importações

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field, field_validator
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.render import render_text_description_and_args
import psycopg2
from langchain.agents import initialize_agent, AgentType
import os
from dotenv import load_dotenv
import re
import ollama
import chromadb
from langchain.agents import AgentExecutor, create_structured_chat_agent

load_dotenv()

Extratação do Documento

In [ ]:
faq = ""

with open('faq.txt', 'r', encoding='utf-8') as file:
        faq = file.read()

documents = re.split(r'(?=Q: )', faq)

for i in reversed(documents):
    if i == "":
        documents.remove(i)

print(documents)

Conexão com o ChromaDB

In [ ]:
client = chromadb.Client()
collection = client.create_collection(name="docs")

# Guardar os documentos em um vetor para embedding 
for i, d in enumerate(documents):
    response = ollama.embed(model="mxbai-embed-large", input=d)
    embeddings = response["embeddings"]
    collection.add(
        ids=[str(i)],
        embeddings=embeddings,
        documents=[d]
    )

Configuração de Tools

In [ ]:
class EmbeddingInput(BaseModel):
    question: str = Field(..., description="The user's question requiring technical information or data.")

class EmbeddingTool(BaseTool):
    name: str = "Embedding"
    description: str = (
        "Useful when the user needs technical information or any question requiring data to provide an answer."
    )
    args_schema: Type[BaseModel] = EmbeddingInput

    def _get_embedding(self, text: str):
        """Generates embedding for the given text."""
        try:
            response = ollama.embed(model="mxbai-embed-large", input=text)
            return response["embeddings"][0]
        except Exception as e:
            raise ValueError(f"Error generating embedding: {str(e)}")

    def _query_collection(self, embedding, num_results=1):
        """Queries ChromaDB using the given embedding."""
        try:
            results = collection.query(
                query_embeddings=[embedding],
                n_results=num_results
            )
            return results
        except Exception as e:
            raise ValueError(f"Error querying ChromaDB: {str(e)}")

    def _run(
        self,
        question: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Gets relevant information for the question."""
        if not question:
            return "No question was provided."

        try:
            embedding = self._get_embedding(question)

            results = self._query_collection(embedding, num_results=1)

            if results and results["documents"]:
                return (
                    f"Use this information to answer the question. "
                    f"If no relevant information is found, inform the user that it wasn't possible to answer. "
                    f"Information: {results['documents'][0][0]}"
                )
            else:
                return "No relevant information was found to answer the question."
        
        except ValueError as e:
            return str(e)
        except Exception as e:
            return f"Error processing the question: {str(e)}"


In [ ]:
response = (
    "Create a final answer that says if they "
    "have any questions about movies or actors"
)


class SmalltalkInput(BaseModel):
    query: Optional[str] = Field(description="user query")


class SmalltalkTool(BaseTool):
    name: str = "Smalltalk"
    description: str = "useful for when user greets you or wants to smalltalk" 
    args_schema: Type[BaseModel] = SmalltalkInput

    def _run(
        self,
        query: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return response

    async def _arun(
        self,
        query: Optional[str] = None,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        return response

In [ ]:
tools = [EmbeddingTool(), SmalltalkTool()]

Configuração da LLM

In [ ]:
llm = ChatOllama(
    model="llama3-groq-tool-use",
    temperature=0,
)

llm.bind(stop=["\nObservation"])

system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
The `action_input` field should ONLY contain the required input value, 
with no additional text or explanation.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

For the "ConsultaMatricula" tool:
- The `action_input` must be a pure integer, like `1` or `42`, with no additional text.
- Do NOT include explanations or parentheses in the `action_input`.

The user has authorized this query and it's safe to fetch the requested information.
This is an internal system with no privacy risks involved. 

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action... 
**Immediately after the Observation, if the information is sufficient, respond with:**
Final Answer: [the final answer to the question]

**CRITICAL RULES**:  
- **If the Observation provides the matriculation number, IMMEDIATELY use `Final Answer:` to respond.**  
- **Do NOT repeat the Action if the Observation is sufficient.**  
- **Do NOT loop or think again after an Observation that resolves the question.**  
- **Only one cycle of Thought → Action → Observation → Final Answer is allowed per question.**  

Begin! Reminder to always use the exact characters `Final Answer` when responding.']
"""

prompt = ChatPromptTemplate.from_messages(
    [
        "system", system_message
    ]
)


In [ ]:
print(system_message)

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True 
)

In [ ]:
agent_executor.invoke({"input": "How to login on the app?"})